In [ ]:
def register_update_tabs_callback(app, name):
    @app.callback(
        Output("file_tabs" + name, "tabs"),
        Output("file_tabs" + name, "children"),
        Input("uploaded-file" + name, "data"),
        Input("file_tabs" + name, "active_tab"),
    )
    def update_tabs(data, active_tab):
        if not data or not data[0]:
            return [], []

        file_urls, file_texts, file_names = data
        tabs = []
        for i, fname in enumerate(file_names):
            tabs.append({"id": f"tab{i+1}", "title": fname})

        if not active_tab:
            active_tab = "tab1" + name
        tab_index_str = active_tab.replace("tab", "").replace(name, "")
        try:
            tab_id = int(tab_index_str) - 1
        except ValueError:
            tab_id = 0

        if tab_id < 0 or tab_id >= len(file_urls):
            return tabs, []

        # If it's a PDF, we have a /pdf/ URL. If it's a TXT, could be None or something else
        file_url = file_urls[tab_id]
        child = []
        if file_url is not None:
            # We'll embed that route in an <iframe> or <object>
            child = [
                html.Iframe(
                    src=file_url,  # e.g. "/pdf/filename.pdf"
                    style={"width": "100%", "height": "1000px"}
                )
            ]

        return tabs, child